In [2]:
from dotenv import load_dotenv
import os

load_dotenv()  # Carregar as variáveis do .env

# Verificar se a chave foi carregada corretamente
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("OPENAI_API_KEY não encontrada no arquivo .env")

In [3]:
texto = """
A Microsoft foi fundada em 4 de abril de 1975 por Bill Gates e Paul Allen, em Albuquerque, 
no estado do Novo México. O nome "Microsoft" é uma combinação das palavras "microcomputador" 
e "software", refletindo o foco da empresa em software para computadores pessoais.
O primeiro grande projeto da Microsoft foi a criação de um sistema operacional para o 
computador Altair 8800, um dos primeiros microcomputadores disponíveis comercialmente. 
O sistema, denominado Altair BASIC, foi desenvolvido em parceria com a MITS 
(Micro Instrumentation and Telemetry Systems) e foi um marco inicial para a Microsoft.
Em 1980, a empresa firmou um contrato significativo com a IBM para fornecer o sistema 
operacional para o novo PC da IBM, o que levou à criação do MS-DOS. Esse contrato foi um 
ponto de virada para a Microsoft, impulsionando sua expansão e dominando o mercado de 
sistemas operacionais para PCs nos anos seguintes.
Com o sucesso do MS-DOS, a Microsoft se consolidou como líder no setor de software e, 
em 1985, lançou o Windows, um sistema operacional gráfico que viria a se tornar a base 
de sua supremacia no mercado de sistemas operacionais para desktop.
"""

In [4]:
from langchain.pydantic_v1 import BaseModel, Field
from typing import List
from langchain_core.utils.function_calling import convert_to_openai_function

In [5]:
class Event(BaseModel):
    """Informações sobre um evento ocorrido"""
    date: str = Field(description="Data do evento no formato YYYY-MM-DD")
    event: str = Field(description="Descrição do evento extraído do texto")
    
class EventsList(BaseModel):
    """Lista de Eventos para Extração"""
    events: List[Event] = Field(description="Conjunto de eventos encontrados no texto fornecido")
    
tool_events = convert_to_openai_function(EventsList)
tool_events

{'name': 'EventsList',
 'description': 'Lista de Eventos para Extração',
 'parameters': {'type': 'object',
  'properties': {'events': {'description': 'Conjunto de eventos encontrados no texto fornecido',
    'type': 'array',
    'items': {'description': 'Informações sobre um evento ocorrido',
     'type': 'object',
     'properties': {'date': {'description': 'Data do evento no formato YYYY-MM-DD',
       'type': 'string'},
      'event': {'description': 'Descrição do evento extraído do texto',
       'type': 'string'}},
     'required': ['date', 'event']}}},
  'required': ['events']}}

In [6]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_messages([
    ("system", "Extraia as frases de acontecimentos e as extraia integralmente"),
    ("user", "{input}")
])

chat = ChatOpenAI()

chain = (prompt | chat.bind(functions=[tool_events],
                            function_call={"name":"EventsList"}))

In [7]:
chain.invoke({"input": texto})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"events":[{"date":"1975-04-04","event":"A Microsoft foi fundada por Bill Gates e Paul Allen em Albuquerque, Novo México."},{"date":"1980","event":"A Microsoft firmou contrato com a IBM para fornecer o sistema operacional para o novo PC, resultando no desenvolvimento do MS-DOS."},{"date":"1985","event":"A Microsoft lançou o Windows, um sistema operacional gráfico que se tornou a base de sua supremacia no mercado de sistemas operacionais para desktop."}]}', 'name': 'EventsList'}}, response_metadata={'token_usage': {'completion_tokens': 114, 'prompt_tokens': 426, 'total_tokens': 540, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-2cd97d97-5c49-4806-84

In [8]:
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser

chain = (prompt 
         | chat.bind(functions=[tool_events], function_call={'name': 'EventsList'})
         | JsonOutputFunctionsParser())

chain.invoke({'input': texto})

{'events': [{'date': '1975-04-04',
   'event': 'A Microsoft foi fundada por Bill Gates e Paul Allen em Albuquerque, Novo México.'},
  {'date': '1980',
   'event': 'A Microsoft firmou um contrato com a IBM para fornecer o sistema operacional para o novo PC, levando à criação do MS-DOS.'},
  {'date': '1985',
   'event': 'A Microsoft lançou o Windows, um sistema operacional gráfico que se tornou a base de sua supremacia no mercado de sistemas operacionais para desktop.'}]}

### Dados da Web

In [10]:
from langchain_community.document_loaders.web_base import WebBaseLoader

loader = WebBaseLoader("https://ge.globo.com")
page = loader.load()
page

[Document(page_content=' ge.globo - É esporte sempre                                                                                                                                                                 TIMES      Série A                                                                                                          Série B                                                                                                          Europa                                                                                                                         \xa0é o seu time do coração?   Sim Não    Obrigado pela participação.  Nós usaremos esta informação para trazer mais novidades para você.     Notícias de esportes personalizadas De 0 a 10, qual a sua satisfação com o conteúdo especialmente selecionado para você? Em uma escala, considere 0: insatisfeito e 10: muito satisfeito.  012345 678910  Concluir     Como podemos tornar sua experiência de consumo de notícias de espo

In [11]:
class BlogPost(BaseModel):
    """Detalhes sobre uma postagem de blog"""
    title: str = Field(description="Título da postagem no blog")
    author: str = Field(description="Nome do autor da postagem no blog")
    
class BlogSite(BaseModel):
    """Conjunto de postagens de blog de um site específico"""
    posts: List[BlogPost] = Field(description="Coleção de postagens de blog do site")
    
tool_blog = convert_to_openai_function(BlogSite)
tool_blog

{'name': 'BlogSite',
 'description': 'Conjunto de postagens de blog de um site específico',
 'parameters': {'type': 'object',
  'properties': {'posts': {'description': 'Coleção de postagens de blog do site',
    'type': 'array',
    'items': {'description': 'Detalhes sobre uma postagem de blog',
     'type': 'object',
     'properties': {'title': {'description': 'Título da postagem no blog',
       'type': 'string'},
      'author': {'description': 'Nome do autor da postagem no blog',
       'type': 'string'}},
     'required': ['title', 'author']}}},
  'required': ['posts']}}

In [12]:
from langchain.output_parsers.openai_functions import JsonKeyOutputFunctionsParser
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_messages([
    ('system', 'Extraia da página os posts do blog com as informações especificadas'),
    ('user', '{input}')
])
chat = ChatOpenAI()
chain = (prompt 
         | chat.bind(functions=[tool_blog], function_call={'name': 'BlogSite'})
         | JsonKeyOutputFunctionsParser(key_name='posts'))

In [13]:
chain.invoke({"input": page})

[{'title': 'AO VIVO: assista à apresentação de Tiquinho Soares',
  'author': 'santos'},
 {'title': 'Presidente diz que Zé Rafael vai passar por cirurgia na coluna',
  'author': 'santos'},
 {'title': 'Fluminense acerta venda de Kauã Elias para o Shakhtar Donetsk',
  'author': 'fluminense'},
 {'title': 'Vitória anuncia chegada do atacante Carlinhos',
  'author': 'fluminense'},
 {'title': 'Grêmio fica perto de atacante belga',
  'author': 'futebol internacional'},
 {'title': 'Cristiano Ronaldo defende Vini Jr. como melhor e critica Bola de Ouro: "Não há credibilidade"',
  'author': 'futebol internacional'},
 {'title': 'Arrascaeta e Bruno Henrique recebem homenagem por recorde de títulos pelo Flamengo',
  'author': 'flamengo'},
 {'title': 'Flamengo preserva titulares para jogo contra a Portuguesa',
  'author': 'flamengo'},
 {'title': 'Flamengo libera Varela para ir ao velório da mãe no Uruguai',
  'author': 'flamengo'},
 {'title': 'Após 1º título em 36 dias, José Boto tem desafio de quebra